### Trabalho Final - Aprendizado de Máquina
- Vinícius Braga Freire - 2020054889

#### 1. Processamento de dados

##### 1.1. Interações

Vamos tratar os dados das interações entre indivíduos. Este dado nos dá a informação em que dia, por quanto tempo e a qual reunião um funcionário participou.

Deste dado, iremos gerar um grafo (matriz de adjacência) que irá indicar quantas vezes uma certa pessoa se encontrou com outra. O valor de quantas vezes uma pessoa se encontrou com outra tem forte correlação com estas pertencerem a um mesmo grupo.

Note que reuniões com mais de 1 segundo serão consideradas, além de que reuniões que tenham tido mais de 10 participantes serão descartadas (pois essas reuniões grandes tendem a ser feitas para a empresa toda/setor inteiro).

In [1]:
import pandas as pd

interacoes_df = pd.read_csv("./data/in/interacoes.csv", encoding="cp860", sep=";")
print(interacoes_df.info())
interacoes_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149006 entries, 0 to 149005
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Data             149006 non-null  object
 1   Matricula        149006 non-null  int64 
 2   Departamento     149006 non-null  object
 3   Nome da Reuni╞o  149006 non-null  object
 4   Nome da Janela   149006 non-null  object
 5   Tempo na Janela  149006 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.8+ MB
None


,Data,Matricula,Departamento,Nome da Reuni╞o,Nome da Janela,Tempo na Janela
0,31/12/2022,10085103,CTN-01,-,dtOthers,00:00:56
1,31/12/2022,10085103,CTN-01,Daniela Cristina Arena Vaz,Daniela Cristina Arena Vaz | Microsoft Teams,00:01:02
2,31/12/2022,10085103,CTN-01,[War Room] - Carga Suprimentos,[War Room] - Carga Suprimentos | Microsoft Teams,00:04:58
3,31/12/2022,10085103,CTN-01,Cutover/Golive Florestal,Cutover/Golive Florestal | Microsoft Teams,00:04:46
4,31/12/2022,10085103,CTN-01,Núcleo Duro do Go Live,Núcleo Duro do Go Live | Microsoft Teams,00:07:52


In [15]:
# Pega todas reuniões
interacoes_grouped = (
    interacoes_df[["Data", "Nome da Reuni╞o", "Matricula"]]
    .groupby(["Data", "Nome da Reuni╞o"])
)

# Pega quantas pessoas foram pra cada reunião e filta as com menos ou com exatamente 10 pessoas
interacoes_lt_10 = interacoes_grouped.filter(lambda g: g.Matricula.count() <= 10)
interacoes_lt_10

,Data,Nome da Reuni╞o,Matricula
1,31/12/2022,Daniela Cristina Arena Vaz,10085103
2,31/12/2022,[War Room] - Carga Suprimentos,10085103
3,31/12/2022,Cutover/Golive Florestal,10085103
4,31/12/2022,Núcleo Duro do Go Live,10085103
5,31/12/2022,War Room - Subida J1BTAX e BRF PLUS,10085103
...,...,...,...
149001,01/07/2022,Tropa de Elite,10079664
149002,01/07/2022,-,10088169
149003,01/07/2022,Talita Freire da Silva,10088169
149004,01/07/2022,Product Analytics - integraç╞o e estrutura da ...,10088169


In [19]:
import networkx as nx

def generate_graph():
    return;    

Procura por todas cliques máximas

In [20]:
teams = list(nx.clique.find_cliques())

TypeError: find_cliques() missing 1 required positional argument: 'G'

##### 1.2. Infos Pessoais

QAAAA

In [ ]:
import pandas as pd

infos_pessoas_df = pd.read_csv("./data/in/infos_pessoas.csv", encoding="cp860")
infos_pessoas_df.head()

#### 2. Treinamento de modelo

#### 3. Resultado e validação de modelo